# **Set according to environment (e.g. local, Google Colab...)**

In [5]:
project_folder = ''

# **Body**

In [6]:
from custom_libraries.image_dataset import *
from custom_libraries.miscellaneous import *
import numpy as np

In [7]:
def create_model(num_units):
    model = tf.keras.Sequential()
    model.add(
        tf.keras.layers.Dense(units=num_units, activation=None, kernel_initializer=tf.keras.initializers.HeNormal, ))
    model.add(tf.keras.layers.LeakyReLU(alpha=.01))
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=[tf.keras.losses.BinaryCrossentropy(name='binary_crossentropy'), 'acc'])

    return model

In [ ]:
bs = 256
trials = 10
epochs = 2000
trees_set = [1, 32]

classes = np.load(project_folder + 'results/classes.npy', allow_pickle=True)
# TODO: verificare l'effetto di patience sul risultato di svhn (rischio underfitting)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=60)

#classes = [classes[-2]]
history = np.zeros((len(classes), trials, len(trees_set), 2))

#history = np.load(project_folder+'results/fcnn_history.npy', allow_pickle=True)
#history[-2,:,:,:] = 0

for j, (t1, t2, ds) in enumerate(classes):

    print(f"Dataset: {ds} / Pair: {t1}-{t2}")

    #if history[j, 0, 0, 0] != 0:
    #    continue

    test_ds = ImageDataset(ds, 'test', data_dir=None, USPS_dir=project_folder + 'USPS/')
    train_ds = ImageDataset(ds, 'train', data_dir=None, USPS_dir=project_folder + 'USPS/')

    for x in [train_ds, test_ds]:
        x.filter(t1, t2, overwrite=True)
        x.shuffle()
        x.normalize()
        if x.images.shape[1:3] == (28, 28):
            x.pad()
        x.vectorize(True)

    X_train, y_train, X_valid, y_valid = train_ds.subset(shard=True, shard_number=trials, validation=True,
                                                         validation_size=len(test_ds.images))
    test_set = tf.data.Dataset.from_tensor_slices((test_ds.images, test_ds.labels)).batch(bs)

    for k, trees in enumerate(trees_set):

        print(f"{trees}-FCNN")

        for i in range(trials):

            if history[j, i, k, 0] != 0:
                continue

            print(f"Trial {i + 1}")

            model = create_model(num_units=2 * trees)

            train_set = tf.data.Dataset.from_tensor_slices((X_train[i], y_train[i])).batch(bs)
            valid_set = tf.data.Dataset.from_tensor_slices((X_valid[i], y_valid[i])).batch(bs)

            with tf.device('/device:GPU:0'):

                fit_history = model.fit(x=train_set, batch_size=bs, epochs=epochs,
                                        validation_data=valid_set, validation_batch_size=bs,
                                        callbacks=[callback], verbose=0)
                print_fit_history(fit_history, epochs)

                evaluate_history = model.evaluate(x=test_set, batch_size=bs, verbose=0)
                print_evaluate_history(evaluate_history)

                history[j, i, k] = evaluate_history[1:]

                np.save(project_folder + 'results/fcnn_history.npy', history,
                        allow_pickle=True)

In [ ]:
# Eseguire analisi statistica per confronto con Jones&Kording, Table 2, righe n-FCNN
print(history)